# Collect EWS Usage Data from Audit Logs

This notebook will collect app registration data from Entra and signin audit logs from Microsoft 365 for applications with EWS permissions. The data will be stored in CSV files in the `$OutputPath` folder and can be easily imported in Excel or Power BI for further analysis.

Included in this repo is a second notebook `Report-EWS-App-Usage.ipynb` that can be used to review the data as well.

In [ ]:
# Script to call Find-EwsUsage.ps1 

# Configure Log Level
$VerbosePreference = "SilentlyContinue"

# Ensure Script can run in the current context
Set-ExecutionPolicy -ExecutionPolicy Bypass -Scope Process -Force

In [ ]:
# Import Utilities Module
Import-Module ./Modules/EwsUtilities.psm1

In [ ]:
# Load configuration
$appSettings = Get-Config

# Override variables if settings were loaded
if ($appSettings) {
    $TenantId = $appSettings.TenantId
    $AuditAppId = $appSettings.AuditAppId
    $AuditAppSecret = $appSettings.AuditAppSecret
    $OutputPath = $appSettings.OutputPath
    Write-Host "Using values from configuration file"
} else {
    $TenantId = "Tenant ID of tenant to analyze"
    $AuditAppId = "Application ID of application with audit permissions"
    $AuditAppSecret= "App Secret of application with audit permissions"
    $OutputPath = "Local path to store output files"
    Write-Host "Using hardcoded values"
}

# Ensure the output directory exists
if (-not (Test-Path -Path $OutputPath)) {
    New-Item -ItemType Directory -Path $OutputPath | Out-Null
}


In [ ]:

# Call Find-EwsUsage.ps1 with the required parameters
Write-Host "Calling Find-EwsUsage.ps1 with AppId, TenantId, and ClientSecret..."
#& $FindEwsUsageScriptPath `
./Scripts/Find-EwsUsage.ps1 `
    -OutputPath $OutputPath `
    -OAuthClientId $AuditAppId `
    -OAuthTenantId $TenantId `
    -OAuthClientSecret (ConvertTo-SecureString -String $AuditAppSecret -AsPlainText -Force) `
    -Operation GetEwsActivity `

# Check if the script executed successfully
if ($LASTEXITCODE -eq 0) {
    Write-Host "Find-EwsUsage.ps1 executed successfully. Check the output at: $OutputPath"
} else {
    Write-Host "Find-EwsUsage.ps1 execution failed. Please check the parameters and try again."
}